In [1]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: C:\Users\WFBFM\Documents\venv\etc\jupyter
    - Validating...
      voila 0.3.6 ok


In [4]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

class rlcs_wordle:
    def __init__(self, file):
        self.file = file
        self.df = []
        self.guess_list = []
        self.guess_df = []
        self.mystery_player = []
        self.mystery_series = []
    
    def setup_df(self):
        self.df = pd.read_csv(self.file, index_col = 'Player')
        self.df = self.df.drop(['Controller'], axis=1)
        self.df.fillna({'Team':'None'}, inplace=True)
        # filter out players with no age
        self.df = self.df.rename(columns={'Born' : 'Age_value', 
                                'Approx. Total Winnings': 'Winnings_value'})
        self.df = self.df[self.df['Age_value'].notnull()]
        self.df = self.df[['Region', 'Team', 'Nationality',
              'Current Role', 'Status', 'Age_value', 'Winnings_value']]
        self.df['RLCS Player'] = self.df.index
        self.df = self.df[['RLCS Player'] + [col for col in self.df.columns if col != 'RLCS Player']]
        
        self.df['Winnings Int'] = self.df.apply(lambda row: winnings_to_int(row), axis=1)
        
        # filter to players with winnings > $1,000
        self.df = self.df[self.df['Winnings Int'] > 1000]
        
    def new_game(self):
        self.guess_list = []
        self.guess_df = self.df.loc[self.guess_list]
        self.mystery_player = self.df.sample().index[0]
        self.mystery_series = self.df.loc[self.mystery_player].copy()

        self.mystery_series['Age'] = str(int(self.mystery_series['Age_value']))
        self.mystery_series['Total Winnings'] = self.mystery_series['Winnings_value']
        
    def new_guess(self, guess, give_up_flag = False):
        if guess in self.df.index:
            self.guess_list.append(guess)
            self.guess_df = self.df.loc[self.guess_list]
            self.guess_df['Age'] = self.guess_df.apply(lambda row: age(row, self.mystery_series), axis=1)
            self.guess_df['Total Winnings'] = self.guess_df.apply(
                lambda row: total_winnings(row, self.mystery_series), axis=1)
            self.show_df(give_up_flag)
        else:
            print('Player ' + guess + " isn't recognised.")
            
    def give_up(self):
        return self.new_guess(self.mystery_player, True)
        
    def show_df(self, give_up_flag):
        recent_guess = self.guess_list[-1]
        guesses_left = 8 - len(self.guess_list)
        if give_up_flag:
            caption_str = "Aww, better luck next time - the mystery player was " + self.mystery_player + '!'
        else:
            if recent_guess == self.mystery_player:
                caption_str = "Correct - it's " + recent_guess + "!  You win with " + str(guesses_left) + ' guesses remaining.'
            else:
                if guesses_left < 0:
                    caption_str = "Not " + recent_guess + " either! You have " + str(guesses_left) + " guesses left, but you can keep going..."
                else:
                    caption_str = "Not " + recent_guess + "! You have " + str(guesses_left) + " guesses left."
        
        display(self.guess_df.style.set_properties(**{'background-color': 'white',
                                                   'text-align':'center', 'min-width': '100px',
                                                  'border': '1px dashed black'})\
            .set_caption(caption_str)\
            .set_table_styles([{"selector": "tr", "props": "line-height: 30px;"},
            {"selector": "td,th", "props": "line-height: inherit; padding: 20;"},
            {"selector": "th", "props": "text-align: center; background-color: silver; border: 1px dashed black;"},
            {"selector": "td.col0", "props": "font-weight: bold"},
            {"selector": "caption", "props": "font-weight: bold; font-size:1.25em"}
                              ])\
            .apply(highlight_equal_to_guess, mystery=self.mystery_series,
                               column=self.guess_df.columns, axis=1)\
            .hide_columns(subset=['Age_value', 'Winnings_value', 'Winnings Int'])\
            .hide_index()\
            .applymap(lambda v: 'background-color: moccasin; color: black;' if '↓' in str(v) else '')\
            .applymap(lambda v: 'background-color: cornflowerblue; color: white;' if '↑' in str(v) else ''))
        
def winnings_to_int(row):
    winnings = str(row['Winnings_value'])
    if winnings[0] == '$':
        winnings = winnings[1:].split(',')
        winnings_str = ''
        for entry in winnings:
            winnings_str += entry
    else:
        winnings_str = '0'
    return int(winnings_str)
            
def age(row, mystery):
    if row['Age_value'] < mystery['Age_value']:
        return str(int(row['Age_value'])) + ' ↑'
    elif row['Age_value'] > mystery['Age_value']:
        return str(int(row['Age_value'])) + ' ↓'
    return str(int(row['Age_value']))

def total_winnings(row, mystery):
    if row['Winnings Int'] < mystery['Winnings Int']:
        return row['Winnings_value'] + ' ↑'
    elif row['Winnings Int'] > mystery['Winnings Int']:
        return row['Winnings_value'] + ' ↓'
    return row['Winnings_value']

def highlight_equal_to_guess(s, mystery, column):
    is_equal = pd.Series(data=False, index=s.index)
    is_equal[column] = s.loc[column] == mystery.loc[column]
    return ['background-color: mediumaquamarine; color: white' if v else '' for v in is_equal]


game = rlcs_wordle('rlcs-data.csv')
game.setup_df()
game.new_game()

# Button to initialise new game
new_game_button = widgets.Button(
                description='New Game',
                tooltip='New Game',
                style={'description_width': 'initial'}
            )

def click_new_game_button(event):
    with out:
        clear_output()
    game.setup_df()
    game.new_game()

new_game_button.on_click(click_new_game_button)

# Searchable dropdown to pick from all players
player_picker = widgets.Combobox(
    placeholder='Enter RLCS player...',
    options=tuple(game.df.index.sort_values()),
    description='Guess Player:',
    ensure_option=True,
    disabled=False,
    style={'description_width': 'initial'}
)

# New guess button
def click_guess_button(event):
    if player_picker.value in game.df.index and player_picker.value not in game.guess_df.index:
        with out:
            clear_output()
            game.new_guess(player_picker.value)

guess_button = widgets.Button(
                description='Guess',
                tooltip='Guess',
                style={'description_width': 'initial'}
            )

guess_button.on_click(click_guess_button)


# Give up button
def click_give_up_button(event):
    with out:
        clear_output()
        game.give_up()

give_up_button = widgets.Button(
                description='Give Up',
                tooltip='Give Up',
                style={'description_width': 'initial'}
            )

give_up_button.on_click(click_give_up_button)

display(widgets.HBox((new_game_button, player_picker, guess_button, give_up_button)))

out = widgets.Output()
display(out)

Output()